In [2]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

# Channel data 

In [3]:
channels=['Fp1','F3','C3','P3','F7', 'T7',  'P7', 'O1', 'F9', 'FT9', 'T9', 'P9', 'FP2','F4', 'C4','P4','F8','T8','P8','O2', 'F10','Ft10','T10','P10','FZ','CZ','OZ','Sp1', 'Sp2', 'EKG']         
channel_dict={}
for i in range(len(channels)):
    channel_dict[channels[i]]=i
Selected_electrod=['Sp1', 'Sp2', 'T8', 'P8']

# Load patient data

In [4]:
def dataload(patient_number, dataset_number):
    try :
        input_folder_path = '/home/maestroj/medical_analysis/eeg_csv'
        input_data_name = 'data{}_{}.csv'.format(patient_number,dataset_number)
        input_data_path = os.path.join(input_folder_path,input_data_name)
        input_file = open(input_data_path,'r',newline='')
        reader = csv.reader(input_file, delimiter=',')
        print("{} is loaded".format(input_data_name))
        temp = []
        for row in reader:
            temp.append(row)
        temp = np.array(temp, dtype=np.float32)
        temp = np.transpose(temp)
        return temp
    except FileNotFoundError:
        print("No such File")
        temp = []
        temp = np.array(temp)
        return temp

# Onset-time data

In [5]:
def seizure_time_parser(onset_times,patient_number,data_set_number):
    temp = str(onset_times[patient_number-1][data_set_number])
    curr_onset_time = []
    while temp.find('/')!=-1:
        curr_onset_time.append(int(temp[0:temp.find('/')]))
        temp=temp[temp.find('/')+1:]
    curr_onset_time.append(int(float(temp)))
    curr_onset_time=np.array(curr_onset_time)
    return curr_onset_time

In [6]:
seizure_file = open('seizure_times.csv','r',newline='')
reader = csv.reader(seizure_file,delimiter=',')
onset_times_temp=[]

for row in reader:
    onset_times_temp.append(row)
    
onset_times = []
total_patient = len(onset_times_temp)
event_number = np.zeros(total_patient,dtype=np.int32)
#number of events of (i+1) patient = event_number[i]

for i in range(total_patient):
    event_number[i] = len(onset_times_temp[i])-1

for p in range(total_patient):
    temp = []
    for d in range(event_number[p]):
        temp.append(seizure_time_parser(onset_times_temp,p+1,d+1))
    onset_times.append(temp)

In [7]:
print("The number of datasets of patients")
print(event_number)
print("1st patient's seizure time for each data_set")
print(onset_times[0])
print("9th patient's seizure time for each data_set")
print(onset_times[8])

The number of datasets of patients
[ 7  7  6  3 10  1  7 13 16  8  8  5  3  6 31  8  2  4]
1st patient's seizure time for each data_set
[array([820]), array([781]), array([626]), array([713]), array([807]), array([792]), array([780])]
9th patient's seizure time for each data_set
[array([200]), array([312]), array([624]), array([762]), array([752]), array([772]), array([884]), array([502]), array([ 94, 258]), array([836]), array([650]), array([636]), array([576]), array([656]), array([638]), array([706])]


# Setting hyper parameters

In [8]:
data_size = 100

input_data_size = data_size*3

error_range = 5
sampling_rate = 200
striding = 100

# Training_set

In [9]:
def downgrade_and_sum_up(temp):
    """
    downgrade with striding information 
    return 3 arrays max, min ,average
    """
    nchannels = temp.shape[0]
    data_length = temp.shape[1]
    reduced_data_length = int(data_length/striding)
    max_array = np.zeros(reduced_data_length)
    min_array = np.zeros(reduced_data_length)
    average_array = np.zeros(reduced_data_length)
    for i in range(reduced_data_length):
        for channel in range(nchannels):
            max_array[i] += np.max(temp[channel][striding*i:striding*(i+1)])
            min_array[i] += np.min(temp[channel][striding*i:striding*(i+1)])
            average_array[i] += np.average(temp[channel][striding*i:striding*(i+1)])
    return {'max' : max_array,'min' : min_array,'average':average_array}

In [10]:
def normalize_max(array):
    temp = np.zeros(len(array),dtype=np.float32)
    min = np.min(array)
    max = np.max(array)
    for i in range(len(array)):
        temp[i]=(array[i]-min)/(max-min)
    return temp

def normalize_min(array):
    temp = np.zeros(len(array),dtype=np.float32)
    min = np.min(array)
    max = np.max(array)
    for i in range(len(array)):
        temp[i]=(max-array[i])/(max-min)
    return temp

def normalize_average(array):
    temp = np.zeros(len(array),dtype=np.float32)
    average = np.average(array)
    min = np.min(array)
    max = np.max(array)
    for i in range(len(array)):
        temp[i]=(array[i]-average)/(max-min)
    return temp

In [14]:
def train_set_generator(patient_number, dataset_number):
    temp = dataload(patient_number, dataset_number)
    onset_times_temp = onset_times[patient_number-1][dataset_number-1]        
    
    result = downgrade_and_sum_up(temp)
    input_data = []
    output_data = []
    
    nchannels = temp.shape[0]
    
    reduced_data_length = len(result['max'])
    
    result['max'] = normalize_max(result['max'])
    result['min'] = normalize_min(result['min'])
    result['average'] = normalize_average(result['average'])
        
    for i in range(reduced_data_length):
        initial_point = i
        final_point = i+data_size 
        mid_point = (initial_point + final_point)/2
        if final_point > reduced_data_length:
            break

        input_temp = []
            
        for k in range(initial_point,final_point):
            input_temp.append(result['max'][k])
            input_temp.append(result['min'][k])
            input_temp.append(result['average'][k])

        output_temp = 0
        for k in range(len(onset_times_temp)):
            position = onset_times_temp[k]*sampling_rate/striding    
            if mid_point < position and position < final_point:
                output_temp = 1
                break;
                
        input_data.append(input_temp)
        output_data.append(output_temp)
       
    input_data = np.array(input_data, dtype =np.float32)
    output_data = np.array(output_data, dtype = np.float32)
    return {'input' : input_data, 'output' : output_data}

result = train_set_generator(1, 1)
input_data = result['input']
output_data =result['output']
output_data = np.reshape(output_data, [-1,1])

print(input_data.shape)
print(output_data.shape)
print(len(temp))

# Neural network

In [12]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, input_data_size])
y = tf.placeholder(tf.float32, shape=[None, 1])

W_fc1 = tf.Variable(tf.truncated_normal([input_data_size, 50], stddev=0.01))
b_fc1 = tf.Variable(tf.constant(0.01, shape=[50]))
h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

W_fc2 = tf.Variable(tf.truncated_normal([50, 50], stddev=0.01))
b_fc2 = tf.Variable(tf.constant(0.01, shape=[50]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

W_fc3 = tf.Variable(tf.truncated_normal([50, 50], stddev=0.01))
b_fc3 = tf.Variable(tf.constant(0.01, shape=[50]))
h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)

W_fc4 = tf.Variable(tf.truncated_normal([50, 50], stddev=0.01))
b_fc4 = tf.Variable(tf.constant(0.01, shape=[50]))
h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_fc4) + b_fc4)

W_fc5 = tf.Variable(tf.truncated_normal([50, 1], stddev=0.01))
b_fc5 = tf.Variable(tf.constant(0.01, shape=[1]))
y_hat = tf.sigmoid(tf.matmul(h_fc4, W_fc5) + b_fc5)

y_predict = tf.floor(y_hat+0.5)

error = tf.reduce_mean((y-y_hat)*(y-y_hat))
train_step = tf.train.AdamOptimizer(1e-4).minimize(error)

sess.run(tf.global_variables_initializer())

In [ ]:
iteration = 100
for k in range(iteration):
    for i in range(total_patient):
        for j in range(event_number[i]):
            result = train_set_generator(i+1, j+1)
            input_data = result['input']
            output_data =result['output']
            output_data = np.reshape(output_data, [-1,1])
            plt.plot(output_data)
            plt.title('Output_data(patient{}, event{})'.format(i+1, j+1))
            plt.show()
            for k in range(100):
                train_step.run(feed_dict={x : input_data, y : output_data})
            print("Error = {}".format(sess.run(error, feed_dict={x: input_data, y: output_data})))
            plt.plot(sess.run(y_hat, feed_dict = {x : input_data}))
            plt.title('Prediction_data(patient{},event{})'.format(i+1, j+1))
            plt.show()

data1_1.csv is loaded


# Test_set_generator

In [ ]:
def test_set_generator(temp,onset_times_temp, channel_number):
    result = downgrade(temp)
    reduced_data_length = result['max'].shape[1]
    input_data = []
    output_data = []
    result['max'][channel_number] = normalize_max(result['max'][channel_number])
    result['min'][channel_number] = normalize_min(result['min'][channel_number])
    result['average'][channel_number] = normalize_average(result['average'][channel_number])
    
    for i in range(reduced_data_length):
        initial_point = i
        final_point = i+data_size
        mid_point = (initial_point+final_point)/2
        
        if final_point > reduced_data_length:
            break
        input_temp = []
        for k in range(initial_point,final_point):
            input_temp.append(result['max'][channel_number][k])
            input_temp.append(result['min'][channel_number][k])
            input_temp.append(result['average'][channel_number][k])
        output_temp = 0

        onset_times_temp = onset_times[patient_number-1][dataset_number-1]
        for k in range(len(onset_times_temp)):
            position = onset_times_temp[k]*sampling_rate/striding
            if mid_point < position and position < final_point:
                output_temp = 1
                break;
        input_data.append(input_temp)
        output_data.append(output_temp) 

    input_data = np.array(input_data, dtype =np.float32)
    output_data = np.array(output_data, dtype = np.float32)
    return {'input' :  input_data, 'output' : output_data}

In [ ]:
patient_number = 9
dataset_number = 5

temp = dataload(patient_number, dataset_number)
nchannels = temp.shape[0]
onset_times_temp = onset_times[patient_number-1][dataset_number-1]

flag = 0

for channel_number in range(nchannels):
    result = test_set_generator(temp,onset_times_temp,channel_number)
    val_input = result['input']
    val_output = result['output']

    if flag==0:
        flag=1
        time_step_size = striding/sampling_rate
        time = np.zeros(val_input.shape[0])
        for i in range(len(time)):
            time[i] = (i + data_size)*time_step_size   
        plt.plot(time, val_output)
        plt.title("Seizure_time_output(patient {}, event {})"
                    .format(patient_number,dataset_number))
        plt.set_xlabel('sec')
        plt.vlines(onset_times[patient_number-1][dataset_number-1],0,1.5, color='r'
                   , alpha=0.9, linestyle='--', label='Onsets')
        plt.show()

    prediction = sess.run(y_predict, feed_dict={x: val_input})
    plt.plot(time, prediction)
    plt.title("Prediction(patient {}, event {}, Channel {})"
                .format(patient_number,dataset_number,channel_number))
    plt.set_xlabel('sec')
    plt.show()